## 実験的にコードを実行したい場合はこのファイルで

In [1]:
import argparse
import pathlib
import numpy as np
import sklearn.model_selection
from torch import utils
import torch
import torchvision
import torchvision.transforms.functional
from torchvision import transforms
import os

In [2]:
data_dir = './dogs-vs-cats-redux-kernels-edition'
train_path = os.path.join(data_dir, "train")

In [3]:
# mpsを利用できるか確認
if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")

else:
    print('mps is available')

mps is available


In [ ]:
def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--data_dir", required=True)
    parser.add_argument("--device", default="mps")
    args = parser.parse_args()

    data_dir = pathlib.Path(args.data_dir)

    print(data_dir)


if __name__ == "__main__":
    main()


In [5]:
def setup_center_crop_transform():
    return transforms.Compose(
        [
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]
    )

In [ ]:
def get_labels(dataset):
    if isinstance(dataset, torch.utils.data.Subset):
        return get_labels(dataset.dataset)[dataset.indices]
    else:
        return np.array([img[1] for img in dataset.imgs])
    

def setup_train_val_datasets(data_dir, dryrun=False):
    dataset = torchvision.datasets.ImageFolder(
        os.path.join(data_dir, "train"),
        transform=setup_center_crop_transform(),
    )
    labels = get_labels(dataset)
    return labels


    

In [8]:
dataset = torchvision.datasets.ImageFolder(
        os.path.join(data_dir, "train"),
        transform=setup_center_crop_transform(),
    )

if isinstance(dataset, torch.utils.data.Subset):
    get_labels(dataset.dataset)[dataset.indices]
else:
    np.array([img[1] for img in dataset.imgs])

In [9]:
len(np.array([img[1] for img in dataset.imgs]))

25000

In [6]:
torchvision.datasets.ImageFolder(
        os.path.join(data_dir, "train"),
        transform=setup_center_crop_transform()
    ).class_to_idx

{'cat': 0, 'dog': 1}

In [ ]:
for img in dataset.imgs:
    print(np.array(img[0]))